In [ ]:
!pip install bert-score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import requests
import os
import json
import csv
import torch
import re

In [ ]:
df = pd.read_csv("SDS.csv")
df.head()

,bug.id,bug_report,golden_summary
0,1,"""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,"""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,"""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,"""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
df = pd.read_csv("ADS.csv")
df.head()

,bug.id,bug_report,golden_summary
0,1,"""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
df2 = pd.read_csv("codellama-br-ft-eval-sds.csv")
df2.head()

,bug.id,summary
0,1,"1. Bug report from Matt N, Justin D, Gavn Shap..."
1,2,Remove the preference to enable session restore.
2,3,"Firefox should disable the ""rotate"" gesture th..."
3,4,The operation CreateResourceOperation#execute(...
4,5,1. Counting lines when applying patch in Eclipse.


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,"1. Bug report from Matt N, Justin D, Gavn Shap...","""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,Remove the preference to enable session restore.,"""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"Firefox should disable the ""rotate"" gesture th...","""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,The operation CreateResourceOperation#execute(...,"""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,1. Counting lines when applying patch in Eclipse.,"""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/792 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.04G [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.55 seconds, 2.66 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5249840021133423, R=0.37675994634628296, F1=0.43868982791900635
Row 1, Bug ID 2: BERTScore P=0.6027543544769287, R=0.29028987884521484, F1=0.3918584883213043
Row 2, Bug ID 3: BERTScore P=0.6152148842811584, R=0.371868371963501, F1=0.46354541182518005
Row 3, Bug ID 4: BERTScore P=0.601680338382721, R=0.4392641484737396, F1=0.5078015327453613
Row 4, Bug ID 5: BERTScore P=0.5310587882995605, R=0.2295699119567871, F1=0.32056406140327454
Row 5, Bug ID 6: BERTScore P=0.5286272168159485, R=0.3608402907848358, F1=0.4289083182811737
Row 6, Bug ID 7: BERTScore P=0.6443493366241455, R=0.36939775943756104, F1=0.46958693861961365
Row 7, Bug ID 8: BERTScore P=0.57021164894104, R=0.3531964123249054, F1=0.4362030625343323
Row 8, Bug ID 9: BERTScore P=0.6209296584129333, R=0.42199453711509705, F1=0.5024889707565308
Row 9, Bug ID 10: BERTScore P=0.576246976852417, R=0.44796663522720337, F1=0.504073441028595
Row 10, Bug ID 11: BERTS

In [ ]:
df2 = pd.read_csv("codellama-br-ft-eval-sds-with-code.csv")
df2.head()

,bug.id,summary
0,1,The bug is caused by the fact that when a user...
1,2,Remove the preference to enable session restore.
2,3,"Firefox should not have a ""rotate"" gesture tha..."
3,4,The Eclipse API Create*Operation classes do no...
4,5,"""Eclipse - Counting Added/Removed/Filtered lines"""


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,The bug is caused by the fact that when a user...,"""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,Remove the preference to enable session restore.,"""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"Firefox should not have a ""rotate"" gesture tha...","""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,The Eclipse API Create*Operation classes do no...,"""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,"""Eclipse - Counting Added/Removed/Filtered lines""","""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.22 seconds, 2.72 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5683434009552002, R=0.39515411853790283, F1=0.4661833345890045
Row 1, Bug ID 2: BERTScore P=0.6027542948722839, R=0.29028987884521484, F1=0.3918584883213043
Row 2, Bug ID 3: BERTScore P=0.5927442312240601, R=0.36573657393455505, F1=0.4523580074310303
Row 3, Bug ID 4: BERTScore P=0.5758825540542603, R=0.41347214579582214, F1=0.48134684562683105
Row 4, Bug ID 5: BERTScore P=0.5238704085350037, R=0.2743944823741913, F1=0.3601490259170532
Row 5, Bug ID 6: BERTScore P=0.5187854766845703, R=0.3707564175128937, F1=0.4324541687965393
Row 6, Bug ID 7: BERTScore P=0.5870910882949829, R=0.3506159782409668, F1=0.43903589248657227
Row 7, Bug ID 8: BERTScore P=0.5871708989143372, R=0.2630258798599243, F1=0.36330682039260864
Row 8, Bug ID 9: BERTScore P=0.5407557487487793, R=0.36690646409988403, F1=0.4371819794178009
Row 9, Bug ID 10: BERTScore P=0.6206097602844238, R=0.3799710273742676, F1=0.471353679895401
Row 10, Bug ID 11: B

In [ ]:
df2 = pd.read_csv("llama3-br-ft-eval-sds.csv")
df2.head()

,bug.id,summary
0,1,he bug was fixed by discarding the incorrect p...
1,2,"Remove the ""browser.sessionrestore.enabled"" pr..."
2,3,Disable the tab-swipe gesture (twist) on all p...
3,4,Folder creation operation fails when folder ex...
4,5,"Provide a way to count added, removed, and fil..."


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,he bug was fixed by discarding the incorrect p...,"""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,"Remove the ""browser.sessionrestore.enabled"" pr...","""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,Disable the tab-swipe gesture (twist) on all p...,"""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,Folder creation operation fails when folder ex...,"""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,"Provide a way to count added, removed, and fil...","""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.49 seconds, 2.67 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.48423629999160767, R=0.4264988601207733, F1=0.4535374343395233
Row 1, Bug ID 2: BERTScore P=0.48402634263038635, R=0.41818395256996155, F1=0.4487026035785675
Row 2, Bug ID 3: BERTScore P=0.4738307297229767, R=0.41974371671676636, F1=0.4451502859592438
Row 3, Bug ID 4: BERTScore P=0.4732729196548462, R=0.42897501587867737, F1=0.45003652572631836
Row 4, Bug ID 5: BERTScore P=0.5569060444831848, R=0.4224473237991333, F1=0.48044654726982117
Row 5, Bug ID 6: BERTScore P=0.45495644211769104, R=0.421813040971756, F1=0.43775829672813416
Row 6, Bug ID 7: BERTScore P=0.5887249112129211, R=0.49296993017196655, F1=0.5366091728210449
Row 7, Bug ID 8: BERTScore P=0.4993930459022522, R=0.38388997316360474, F1=0.4340895712375641
Row 8, Bug ID 9: BERTScore P=0.5045725703239441, R=0.4774262309074402, F1=0.49062418937683105
Row 9, Bug ID 10: BERTScore P=0.48633989691734314, R=0.4086446464061737, F1=0.4441198408603668
Row 10, Bug ID 

In [ ]:
df2 = pd.read_csv("llama3-br-ft-eval-sds-with-code.csv")
df2.head()

,bug.id,summary
0,1,The bug is about the incorrect previous search...
1,2,"Remove the ""browser.sessionrestore.enabled"" pr..."
2,3,"isable the ""rotate"" tab-swapping gesture\n ..."
3,4,Folder creation and file creation operations d...
4,5,Provide a way to count the lines changed by an...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,The bug is about the incorrect previous search...,"""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,"Remove the ""browser.sessionrestore.enabled"" pr...","""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"isable the ""rotate"" tab-swapping gesture\n ...","""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,Folder creation and file creation operations d...,"""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,Provide a way to count the lines changed by an...,"""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.37 seconds, 2.69 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5141323208808899, R=0.4461299180984497, F1=0.47772327065467834
Row 1, Bug ID 2: BERTScore P=0.5221747756004333, R=0.3429226875305176, F1=0.41397780179977417
Row 2, Bug ID 3: BERTScore P=0.47599631547927856, R=0.43037259578704834, F1=0.45203617215156555
Row 3, Bug ID 4: BERTScore P=0.4782840609550476, R=0.4220970571041107, F1=0.4484374225139618
Row 4, Bug ID 5: BERTScore P=0.5609188079833984, R=0.4148808717727661, F1=0.476971834897995
Row 5, Bug ID 6: BERTScore P=0.46953147649765015, R=0.44303199648857117, F1=0.4558970034122467
Row 6, Bug ID 7: BERTScore P=0.5561727285385132, R=0.4809844493865967, F1=0.515853226184845
Row 7, Bug ID 8: BERTScore P=0.39760535955429077, R=0.3600764572620392, F1=0.3779114782810211
Row 8, Bug ID 9: BERTScore P=0.45662760734558105, R=0.42912864685058594, F1=0.44245126843452454
Row 9, Bug ID 10: BERTScore P=0.5585695505142212, R=0.4348856806755066, F1=0.4890283942222595
Row 10, Bug ID 11:

In [ ]:
df2 = pd.read_csv("mistral-br-ft-eval-sds.csv")
df2.head()

,bug.id,summary
0,1,Search suggestions passing wrong results to Fo...
1,2,Removal of browser session store preference ca...
2,3,"""Accidentally triggers tab rotation gesture wh..."
3,4,"""Eclipse Create Existent Resource From Handle ..."
4,5,"""Provide line count information when applying ..."


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,Search suggestions passing wrong results to Fo...,"""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,Removal of browser session store preference ca...,"""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"""Accidentally triggers tab rotation gesture wh...","""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,"""Eclipse Create Existent Resource From Handle ...","""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,"""Provide line count information when applying ...","""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.26 seconds, 2.72 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5488419532775879, R=0.3022061884403229, F1=0.3897862732410431
Row 1, Bug ID 2: BERTScore P=0.6518567800521851, R=0.3377208709716797, F1=0.44492849707603455
Row 2, Bug ID 3: BERTScore P=0.5932919979095459, R=0.34466809034347534, F1=0.4360288083553314
Row 3, Bug ID 4: BERTScore P=0.5003271102905273, R=0.28284645080566406, F1=0.36139050126075745
Row 4, Bug ID 5: BERTScore P=0.48520663380622864, R=0.22351223230361938, F1=0.3060441017150879
Row 5, Bug ID 6: BERTScore P=0.5681607127189636, R=0.37479496002197266, F1=0.4516517221927643
Row 6, Bug ID 7: BERTScore P=0.4967714548110962, R=0.3559080958366394, F1=0.41470441222190857
Row 7, Bug ID 8: BERTScore P=0.5592424869537354, R=0.27639591693878174, F1=0.3699502944946289
Row 8, Bug ID 9: BERTScore P=0.6177960634231567, R=0.3951781392097473, F1=0.48202505707740784
Row 9, Bug ID 10: BERTScore P=0.6026738882064819, R=0.33129194378852844, F1=0.42755526304244995
Row 10, Bug ID 

In [ ]:
df2 = pd.read_csv("mistral-br-ft-eval-sds-with-code.csv")
df2.head()

,bug.id,summary
0,1,"""Firefox: Search suggestions passing wrong res..."
1,2,Removing the 'browser. session store. enabled'...
2,3,"""Firefox should disable the 'rotate' gesture u..."
3,4,"""CreateExistantResourcefromHandle method in Ec..."
4,5,"""Provide a line count for changes made by the ..."


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,"""Firefox: Search suggestions passing wrong res...","""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,Removing the 'browser. session store. enabled'...,"""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"""Firefox should disable the 'rotate' gesture u...","""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,"""CreateExistantResourcefromHandle method in Ec...","""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,"""Provide a line count for changes made by the ...","""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.24 seconds, 2.72 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.523858904838562, R=0.32145875692367554, F1=0.398427814245224
Row 1, Bug ID 2: BERTScore P=0.575861394405365, R=0.31847554445266724, F1=0.4101312756538391
Row 2, Bug ID 3: BERTScore P=0.5911145806312561, R=0.38305625319480896, F1=0.46486741304397583
Row 3, Bug ID 4: BERTScore P=0.5909550189971924, R=0.3643326163291931, F1=0.4507630467414856
Row 4, Bug ID 5: BERTScore P=0.5275835394859314, R=0.2532275915145874, F1=0.3422049582004547
Row 5, Bug ID 6: BERTScore P=0.6082316040992737, R=0.4053739011287689, F1=0.48650333285331726
Row 6, Bug ID 7: BERTScore P=0.5960801839828491, R=0.39816561341285706, F1=0.4774244725704193
Row 7, Bug ID 8: BERTScore P=0.529832124710083, R=0.29313355684280396, F1=0.3774436116218567
Row 8, Bug ID 9: BERTScore P=0.6958032250404358, R=0.4423133134841919, F1=0.5408286452293396
Row 9, Bug ID 10: BERTScore P=0.5164424180984497, R=0.326294481754303, F1=0.39991676807403564
Row 10, Bug ID 11: BERTS

In [ ]:
df2 = pd.read_csv("phi-3-br-ft-eval-sds.csv")
df2.head()

,bug.id,summary
0,1,Firefox bug - incorrect previous search result...
1,2,Firefox session restore disabled bug fix. \nBa...
2,3,"Disabling 'multitap' rotate ""discovery"" for Fi..."
3,4,Eclipse creates existing resources when creati...
4,5,A patch to improve the Add/Remove line count f...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,Firefox bug - incorrect previous search result...,"""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,Firefox session restore disabled bug fix. \nBa...,"""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"Disabling 'multitap' rotate ""discovery"" for Fi...","""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,Eclipse creates existing resources when creati...,"""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,A patch to improve the Add/Remove line count f...,"""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.55 seconds, 2.66 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.4974548816680908, R=0.40046426653862, F1=0.4437212347984314
Row 1, Bug ID 2: BERTScore P=0.4601917564868927, R=0.3710964322090149, F1=0.4108695685863495
Row 2, Bug ID 3: BERTScore P=0.36253970861434937, R=0.41194701194763184, F1=0.38566744327545166
Row 3, Bug ID 4: BERTScore P=0.48265713453292847, R=0.35017359256744385, F1=0.40587788820266724
Row 4, Bug ID 5: BERTScore P=0.47080522775650024, R=0.24419140815734863, F1=0.32158640027046204
Row 5, Bug ID 6: BERTScore P=0.5906425714492798, R=0.4183497726917267, F1=0.4897860586643219
Row 6, Bug ID 7: BERTScore P=0.6011707782745361, R=0.48054349422454834, F1=0.5341312289237976
Row 7, Bug ID 8: BERTScore P=0.45881253480911255, R=0.34806180000305176, F1=0.39583641290664673
Row 8, Bug ID 9: BERTScore P=0.5049940347671509, R=0.4544084072113037, F1=0.4783676266670227
Row 9, Bug ID 10: BERTScore P=0.52855384349823, R=0.36785754561424255, F1=0.4338019788265228
Row 10, Bug ID 11

In [ ]:
df2 = pd.read_csv("phi-3-br-ft-eval-sds-with-code.csv")
df2.head()

,bug.id,summary
0,1,Firefox bug: Search suggestions pass incorrect...
1,2,Remove SessionRestore for better extension com...
2,3,"Disabling 'multitap' rotate ""discovery"" for Fi..."
3,4,Eclipse bug 260276 - CreateExistingResourceFro...
4,5,The patch counting mechanism in Eclipse is bei...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,Firefox bug: Search suggestions pass incorrect...,"""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,Remove SessionRestore for better extension com...,"""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"Disabling 'multitap' rotate ""discovery"" for Fi...","""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,Eclipse bug 260276 - CreateExistingResourceFro...,"""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,The patch counting mechanism in Eclipse is bei...,"""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
print(df2.shape)


(36, 4)


In [ ]:
print(df2.columns.tolist())


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

KeyError: ['summary']

In [ ]:
df2 = pd.read_csv("gemma-br-ft-eval-sds.csv")
df2.head()

,bug.id,summary
0,1,"Sure, Here is a summary in one sentence :\n\nT..."
1,2,"Sure, here is a summary in one sentence :\n\nT..."
2,3,"Sure, Here is the summary:\n\nThe bug reports ..."
3,4,"Sure, here is a summary in one sentence: Creat..."
4,5,"Sure, Here is the summary in one sentence :\n\..."


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,"Sure, Here is a summary in one sentence :\n\nT...","""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,"Sure, here is a summary in one sentence :\n\nT...","""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"Sure, Here is the summary:\n\nThe bug reports ...","""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,"Sure, here is a summary in one sentence: Creat...","""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,"Sure, Here is the summary in one sentence :\n\...","""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.29 seconds, 2.71 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5465366840362549, R=0.3888471722602844, F1=0.45440006256103516
Row 1, Bug ID 2: BERTScore P=0.5674854516983032, R=0.3579409718513489, F1=0.4389896094799042
Row 2, Bug ID 3: BERTScore P=0.6089315414428711, R=0.5152128338813782, F1=0.5581656098365784
Row 3, Bug ID 4: BERTScore P=0.5154295563697815, R=0.38583284616470337, F1=0.4413135349750519
Row 4, Bug ID 5: BERTScore P=0.5914945602416992, R=0.37290245294570923, F1=0.4574252665042877
Row 5, Bug ID 6: BERTScore P=0.5517008304595947, R=0.44511616230010986, F1=0.4927102029323578
Row 6, Bug ID 7: BERTScore P=0.5536082983016968, R=0.4155689477920532, F1=0.47475820779800415
Row 7, Bug ID 8: BERTScore P=0.5375601053237915, R=0.3281501531600952, F1=0.4075276553630829
Row 8, Bug ID 9: BERTScore P=0.5698472857475281, R=0.4400842785835266, F1=0.4966293275356293
Row 9, Bug ID 10: BERTScore P=0.5554746389389038, R=0.36670905351638794, F1=0.4417722523212433
Row 10, Bug ID 11: BE

In [ ]:
df2 = pd.read_csv("gemma-br-ft-eval-sds-with-code.csv")
df2.head()

,bug.id,summary
0,1,"Sure, Here is a summary in one sentence :\n\nT..."
1,2,"Remove the ""browser.Sessionstore .enabled"" pre..."
2,3,"The bug reports describe an issue with the ""ro..."
3,4,"Sure, here is a summary in one sentence :\n\nT..."
4,5,"Sure, here summary in one sentence :\n\nThe pa..."


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,"Sure, Here is a summary in one sentence :\n\nT...","""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,"Remove the ""browser.Sessionstore .enabled"" pre...","""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"The bug reports describe an issue with the ""ro...","""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,"Sure, here is a summary in one sentence :\n\nT...","""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,"Sure, here summary in one sentence :\n\nThe pa...","""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.28 seconds, 2.71 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5666941404342651, R=0.3937686085700989, F1=0.4646642804145813
Row 1, Bug ID 2: BERTScore P=0.6115227937698364, R=0.3762620687484741, F1=0.46587643027305603
Row 2, Bug ID 3: BERTScore P=0.6291896104812622, R=0.4099864363670349, F1=0.49646878242492676
Row 3, Bug ID 4: BERTScore P=0.563840389251709, R=0.406266450881958, F1=0.4722560942173004
Row 4, Bug ID 5: BERTScore P=0.5408940315246582, R=0.29338502883911133, F1=0.38042476773262024
Row 5, Bug ID 6: BERTScore P=0.5472073554992676, R=0.4109780788421631, F1=0.4694085717201233
Row 6, Bug ID 7: BERTScore P=0.5560767650604248, R=0.4278354048728943, F1=0.4835987091064453
Row 7, Bug ID 8: BERTScore P=0.5306181907653809, R=0.34777122735977173, F1=0.4201638698577881
Row 8, Bug ID 9: BERTScore P=0.5710318088531494, R=0.4306359589099884, F1=0.49099481105804443
Row 9, Bug ID 10: BERTScore P=0.5447112917900085, R=0.3553204834461212, F1=0.43008944392204285
Row 10, Bug ID 11: BER

In [ ]:
df2 = pd.read_csv("qwen-br-ft-eval-sds.csv")
df2.head()

,bug.id,summary
0,1,"""Firefox - Search Suggestions Passes Wrong Pre..."
1,2,The core problem is the removal of a preferenc...
2,3,"""Firefox allows users to switch between tabs u..."
3,4,The issue is a bug in Eclipse's API where cert...
4,5,The user is requesting to add a feature that a...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,"""Firefox - Search Suggestions Passes Wrong Pre...","""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,The core problem is the removal of a preferenc...,"""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"""Firefox allows users to switch between tabs u...","""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,The issue is a bug in Eclipse's API where cert...,"""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,The user is requesting to add a feature that a...,"""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.33 seconds, 2.70 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5253380537033081, R=0.4587351083755493, F1=0.4897827208042145
Row 1, Bug ID 2: BERTScore P=0.5084934234619141, R=0.43965786695480347, F1=0.4715769290924072
Row 2, Bug ID 3: BERTScore P=0.5741671323776245, R=0.46608948707580566, F1=0.5145139098167419
Row 3, Bug ID 4: BERTScore P=0.534504771232605, R=0.4561326205730438, F1=0.492218554019928
Row 4, Bug ID 5: BERTScore P=0.5707519054412842, R=0.42874211072921753, F1=0.48965850472450256
Row 5, Bug ID 6: BERTScore P=0.5105627775192261, R=0.4848995804786682, F1=0.4974003732204437
Row 6, Bug ID 7: BERTScore P=0.6092514395713806, R=0.5058866739273071, F1=0.5527785420417786
Row 7, Bug ID 8: BERTScore P=0.5391168594360352, R=0.43044817447662354, F1=0.4786927103996277
Row 8, Bug ID 9: BERTScore P=0.6244348883628845, R=0.5433669686317444, F1=0.5810871124267578
Row 9, Bug ID 10: BERTScore P=0.511920154094696, R=0.4234086275100708, F1=0.46347641944885254
Row 10, Bug ID 11: BERTS

In [ ]:
df2 = pd.read_csv("qwen-br-ft-eval-sds-with-code.csv")
df2.head()

,bug.id,summary
0,1,"""Firefox - Search suggestions pass wrong form ..."
1,2,The core problem is the removal of a preferenc...
2,3,The user is experiencing an issue with the mul...
3,4,The issue is a bug in Eclipse's API where crea...
4,5,The user wants to add a feature to Eclipse tha...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,"""Firefox - Search suggestions pass wrong form ...","""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,The core problem is the removal of a preferenc...,"""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,The user is experiencing an issue with the mul...,"""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,The issue is a bug in Eclipse's API where crea...,"""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,The user wants to add a feature to Eclipse tha...,"""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.35 seconds, 2.70 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5713421106338501, R=0.47485584020614624, F1=0.5186497569084167
Row 1, Bug ID 2: BERTScore P=0.5221354961395264, R=0.44656094908714294, F1=0.48140016198158264
Row 2, Bug ID 3: BERTScore P=0.5270664095878601, R=0.4651773273944855, F1=0.4941917359828949
Row 3, Bug ID 4: BERTScore P=0.5351516008377075, R=0.457192063331604, F1=0.4931095540523529
Row 4, Bug ID 5: BERTScore P=0.5796135067939758, R=0.42367151379585266, F1=0.48952335119247437
Row 5, Bug ID 6: BERTScore P=0.5893977880477905, R=0.5488946437835693, F1=0.5684255957603455
Row 6, Bug ID 7: BERTScore P=0.5648471713066101, R=0.4752331078052521, F1=0.5161795020103455
Row 7, Bug ID 8: BERTScore P=0.49753040075302124, R=0.40862980484962463, F1=0.44871920347213745
Row 8, Bug ID 9: BERTScore P=0.6421886682510376, R=0.5537354946136475, F1=0.5946909785270691
Row 9, Bug ID 10: BERTScore P=0.5028611421585083, R=0.4279336929321289, F1=0.46238166093826294
Row 10, Bug ID 11: 

In [ ]:
df2 = pd.read_csv("deepseek-br-ft-eval-sds.csv")
df2.head()

,bug.id,summary
0,1,Search suggestions are passing incorrect previ...
1,2,The session restore functionality is being rep...
2,3,The Firefox browser has a core bug where the '...
3,4,The createResource methods in eclipse forget t...
4,5,The core problem is the lack of a feature to c...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,Search suggestions are passing incorrect previ...,"""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,The session restore functionality is being rep...,"""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,The Firefox browser has a core bug where the '...,"""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,The createResource methods in eclipse forget t...,"""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,The core problem is the lack of a feature to c...,"""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.33 seconds, 2.70 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.4670078754425049, R=0.4044126272201538, F1=0.43346211314201355
Row 1, Bug ID 2: BERTScore P=0.5492920279502869, R=0.4503213167190552, F1=0.4949071705341339
Row 2, Bug ID 3: BERTScore P=0.5195095539093018, R=0.484346479177475, F1=0.5013121366500854
Row 3, Bug ID 4: BERTScore P=0.48750820755958557, R=0.42454463243484497, F1=0.4538530707359314
Row 4, Bug ID 5: BERTScore P=0.5719685554504395, R=0.40151166915893555, F1=0.4718165695667267
Row 5, Bug ID 6: BERTScore P=0.5066574215888977, R=0.4765531122684479, F1=0.4911443889141083
Row 6, Bug ID 7: BERTScore P=0.4764087200164795, R=0.41111916303634644, F1=0.44136250019073486
Row 7, Bug ID 8: BERTScore P=0.5025510787963867, R=0.3594784140586853, F1=0.4191417396068573
Row 8, Bug ID 9: BERTScore P=0.5627310276031494, R=0.5311249494552612, F1=0.546471357345581
Row 9, Bug ID 10: BERTScore P=0.5361723899841309, R=0.42822542786598206, F1=0.4761575758457184
Row 10, Bug ID 11: BER

In [ ]:
df2 = pd.read_csv("deepseek-br-ft-eval-sds-with-code.csv")
df2.head()

,bug.id,summary
0,1,Search suggestions passing wrong results to hi...
1,2,The session restore functionality of Firefox c...
2,3,The Firefox browser has an issue with the rota...
3,4,The createResource methods in eclipse forget t...
4,5,The core problem is the lack of a built-in way...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,Search suggestions passing wrong results to hi...,"""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,The session restore functionality of Firefox c...,"""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,The Firefox browser has an issue with the rota...,"""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,The createResource methods in eclipse forget t...,"""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,The core problem is the lack of a built-in way...,"""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.31 seconds, 2.70 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.45706966519355774, R=0.39372652769088745, F1=0.42304009199142456
Row 1, Bug ID 2: BERTScore P=0.5374333262443542, R=0.4600675702095032, F1=0.49575021862983704
Row 2, Bug ID 3: BERTScore P=0.5524898767471313, R=0.4940475821495056, F1=0.521636962890625
Row 3, Bug ID 4: BERTScore P=0.5585795640945435, R=0.4675981104373932, F1=0.5090556144714355
Row 4, Bug ID 5: BERTScore P=0.5229177474975586, R=0.38412171602249146, F1=0.4429003894329071
Row 5, Bug ID 6: BERTScore P=0.5542355179786682, R=0.505802571773529, F1=0.5289126038551331
Row 6, Bug ID 7: BERTScore P=0.494550883769989, R=0.4267588257789612, F1=0.4581606984138489
Row 7, Bug ID 8: BERTScore P=0.4959545135498047, R=0.3625563681125641, F1=0.4188915193080902
Row 8, Bug ID 9: BERTScore P=0.4253777265548706, R=0.4293021559715271, F1=0.42733094096183777
Row 9, Bug ID 10: BERTScore P=0.5538179874420166, R=0.43111538887023926, F1=0.4848235845565796
Row 10, Bug ID 11: BERT

In [ ]:
df2 = pd.read_csv("gpt3.5_eval_sds.csv")
df2.head()

,bug.id,summary
0,1,Firefox search suggestions passing wrong previ...
1,2,"The ""browser.sessionstore.enabled"" preference ..."
2,3,"The core issue is that the multitouch ""rotate""..."
3,4,"""CreateExistentResourceFromHandle"" method does..."
4,5,The patch feature in Eclipse should provide an...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,Firefox search suggestions passing wrong previ...,"""(495584) Firefox - search suggestions passes ...","When typing in the search box, sometimes searc..."
1,2,"The ""browser.sessionstore.enabled"" preference ...","""(449596) Firefox - remove the browser.sessio...",That pref was thought to be for extensions whi...
2,3,"The core issue is that the multitouch ""rotate""...","""(491925) Firefox - Disable multitouch \""rotat...",I've noticed that I frequently trigger the rot...
3,4,"""CreateExistentResourceFromHandle"" method does...","""(250125) Eclipse - createExistentResourceFrom...",The method FolderDescription#createExistentRes...
4,5,The patch feature in Eclipse should provide an...,"""(224588) Eclipse - [Patch] Provide an informa...",Inspired by Martin Oberhuber's mail about his ...


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/2 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1 [00:00<?, ?it/s]

done in 13.25 seconds, 2.72 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5570518970489502, R=0.308178186416626, F1=0.3968221843242645
Row 1, Bug ID 2: BERTScore P=0.6075952053070068, R=0.3811691403388977, F1=0.468456506729126
Row 2, Bug ID 3: BERTScore P=0.6481307744979858, R=0.4568740427494049, F1=0.5359508395195007
Row 3, Bug ID 4: BERTScore P=0.6344295740127563, R=0.3570893108844757, F1=0.456971675157547
Row 4, Bug ID 5: BERTScore P=0.5248175859451294, R=0.28180450201034546, F1=0.36670446395874023
Row 5, Bug ID 6: BERTScore P=0.6320787668228149, R=0.385931134223938, F1=0.479246586561203
Row 6, Bug ID 7: BERTScore P=0.6305723786354065, R=0.38758575916290283, F1=0.48008427023887634
Row 7, Bug ID 8: BERTScore P=0.5571887493133545, R=0.2745719850063324, F1=0.3678663969039917
Row 8, Bug ID 9: BERTScore P=0.697942316532135, R=0.44627895951271057, F1=0.5444348454475403
Row 9, Bug ID 10: BERTScore P=0.6010208129882812, R=0.31453508138656616, F1=0.41295596957206726
Row 10, Bug ID 11: BERTSco

In [ ]:
df2 = pd.read_csv("gpt3.5_eval_ads.csv")
df2.head()

,bug.id,summary
0,1,Javadoc comments incorrectly suggest using dep...
1,2,"Class files not showing when inspecting code, ..."
2,3,Runtime compatibility issue between WTP 1.5.x ...
3,4,Legacy calls create modulefile without workspa...
4,5,Deadlock occurs in Eclipse while changing runt...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,Javadoc comments incorrectly suggest using dep...,"""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,"Class files not showing when inspecting code, ...","""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,Runtime compatibility issue between WTP 1.5.x ...,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,Legacy calls create modulefile without workspa...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,Deadlock occurs in Eclipse while changing runt...,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 8.87 seconds, 10.82 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.6285901069641113, R=0.3982406258583069, F1=0.4875781834125519
Row 1, Bug ID 2: BERTScore P=0.6241706609725952, R=0.37584832310676575, F1=0.46917811036109924
Row 2, Bug ID 3: BERTScore P=0.6499595642089844, R=0.3803964853286743, F1=0.4799163043498993
Row 3, Bug ID 4: BERTScore P=0.6314049363136292, R=0.4039318561553955, F1=0.4926794767379761
Row 4, Bug ID 5: BERTScore P=0.6274623870849609, R=0.41571131348609924, F1=0.5000954866409302
Row 5, Bug ID 6: BERTScore P=0.6391717195510864, R=0.40898397564888, F1=0.4988018274307251
Row 6, Bug ID 7: BERTScore P=0.6508004665374756, R=0.3741198778152466, F1=0.47511476278305054
Row 7, Bug ID 8: BERTScore P=0.568044900894165, R=0.31751301884651184, F1=0.4073401987552643
Row 8, Bug ID 9: BERTScore P=0.5604124069213867, R=0.2876380681991577, F1=0.3801564872264862
Row 9, Bug ID 10: BERTScore P=0.6221998333930969, R=0.36885225772857666, F1=0.46314379572868347
Row 10, Bug ID 11: BERT

In [ ]:

df2 = pd.read_csv("codellama-br-ft-eval-ads.csv")
df2.head()

,bug.id,summary
0,1,"The method getResourceString(Bundle, String) i..."
1,2,When debugging a Java project that references ...
2,3,The bug is caused by the fact that the name of...
3,4,The old way of creating a module file was to c...
4,5,Deadlock in Facet Project Framework.


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,"The method getResourceString(Bundle, String) i...","""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,When debugging a Java project that references ...,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,The bug is caused by the fact that the name of...,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,The old way of creating a module file was to c...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,Deadlock in Facet Project Framework.,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 8.74 seconds, 10.98 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.6311599016189575, R=0.4048558473587036, F1=0.49329128861427307
Row 1, Bug ID 2: BERTScore P=0.6241917610168457, R=0.43689724802970886, F1=0.5140146613121033
Row 2, Bug ID 3: BERTScore P=0.5927044749259949, R=0.45385080575942993, F1=0.5140662789344788
Row 3, Bug ID 4: BERTScore P=0.6330893039703369, R=0.4931524395942688, F1=0.5544272661209106
Row 4, Bug ID 5: BERTScore P=0.6067562103271484, R=0.36953872442245483, F1=0.45932823419570923
Row 5, Bug ID 6: BERTScore P=0.6807433366775513, R=0.3788102865219116, F1=0.4867570102214813
Row 6, Bug ID 7: BERTScore P=0.5735878348350525, R=0.31269747018814087, F1=0.40474429726600647
Row 7, Bug ID 8: BERTScore P=0.5954802632331848, R=0.37481820583343506, F1=0.4600581228733063
Row 8, Bug ID 9: BERTScore P=0.6290645599365234, R=0.29048794507980347, F1=0.3974447548389435
Row 9, Bug ID 10: BERTScore P=0.6244732141494751, R=0.3817017078399658, F1=0.47379931807518005
Row 10, Bug ID 11

In [ ]:
df2 = pd.read_csv("codellama-br-ft-eval-ads-with-code.csv")
df2.head()

,bug.id,summary
0,1,Incorrect Deprecation Comments in Platform Class
1,2,The java source file is not shown when debuggi...
2,3,The runtime id is not the same as the name.
3,4,Eclipse - FlatComponentDeployable Legacy Calls...
4,5,Eclipse - Deadlock while changing runtime in a...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,Incorrect Deprecation Comments in Platform Class,"""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,The java source file is not shown when debuggi...,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,The runtime id is not the same as the name.,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,Eclipse - FlatComponentDeployable Legacy Calls...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,Eclipse - Deadlock while changing runtime in a...,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 8.78 seconds, 10.93 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.570690929889679, R=0.3319472670555115, F1=0.4197458028793335
Row 1, Bug ID 2: BERTScore P=0.6152768135070801, R=0.3776964247226715, F1=0.46806469559669495
Row 2, Bug ID 3: BERTScore P=0.6137478351593018, R=0.32652726769447327, F1=0.4262697398662567
Row 3, Bug ID 4: BERTScore P=0.5308288335800171, R=0.3398362398147583, F1=0.41438406705856323
Row 4, Bug ID 5: BERTScore P=0.6518439650535583, R=0.413620263338089, F1=0.5061002373695374
Row 5, Bug ID 6: BERTScore P=0.680743396282196, R=0.37881025671958923, F1=0.4867570102214813
Row 6, Bug ID 7: BERTScore P=0.6493969559669495, R=0.44283953309059143, F1=0.5265867710113525
Row 7, Bug ID 8: BERTScore P=0.5570865273475647, R=0.40348929166793823, F1=0.4680077135562897
Row 8, Bug ID 9: BERTScore P=0.6314014196395874, R=0.29201844334602356, F1=0.3993435204029083
Row 9, Bug ID 10: BERTScore P=0.6244732141494751, R=0.3817017078399658, F1=0.47379931807518005
Row 10, Bug ID 11: BER

In [ ]:
df2 = pd.read_csv("llama3-br-ft-eval-ads.csv")
df2.head()

,bug.id,summary
0,1,ncorrect comments about deprecations in Platfo...
1,2,ails for external archives in debug mode. \n\n...
2,3,atibility issue between WTP versions. \n\n\n\n...
3,4,ntDeployable creates ModuleFile with wrong typ...
4,5,project framework causes deadlock while changi...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,ncorrect comments about deprecations in Platfo...,"""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,ails for external archives in debug mode. \n\n...,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,atibility issue between WTP versions. \n\n\n\n...,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,ntDeployable creates ModuleFile with wrong typ...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,project framework causes deadlock while changi...,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 9.48 seconds, 10.12 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.49522173404693604, R=0.45918822288513184, F1=0.4765247702598572
Row 1, Bug ID 2: BERTScore P=0.4401370882987976, R=0.4068722724914551, F1=0.42285147309303284
Row 2, Bug ID 3: BERTScore P=0.4876677691936493, R=0.42571067810058594, F1=0.4545878469944
Row 3, Bug ID 4: BERTScore P=0.48287051916122437, R=0.3744449019432068, F1=0.42180135846138
Row 4, Bug ID 5: BERTScore P=0.4457643926143646, R=0.430935800075531, F1=0.4382247030735016
Row 5, Bug ID 6: BERTScore P=0.48026424646377563, R=0.4484245181083679, F1=0.4637985825538635
Row 6, Bug ID 7: BERTScore P=0.5869200229644775, R=0.5278438329696655, F1=0.5558165907859802
Row 7, Bug ID 8: BERTScore P=0.5243349075317383, R=0.41686224937438965, F1=0.4644625782966614
Row 8, Bug ID 9: BERTScore P=0.49413350224494934, R=0.3224603235721588, F1=0.39025139808654785
Row 9, Bug ID 10: BERTScore P=0.5654488801956177, R=0.4788530766963959, F1=0.5185606479644775
Row 10, Bug ID 11: BERTS

In [ ]:
df2 = pd.read_csv("llama3-br-ft-eval-ads-with-code.csv")
df2.head()

,bug.id,summary
0,1,correct comments about deprecations in Platfor...
1,2,when stepping into a variable from an external...
2,3,atibility issue between WTP versions. \n\n\n\n...
3,4,ntDeployable creates ModuleFile with wrong res...
4,5,k in Faceted Project Framework while changing ...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,correct comments about deprecations in Platfor...,"""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,when stepping into a variable from an external...,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,atibility issue between WTP versions. \n\n\n\n...,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,ntDeployable creates ModuleFile with wrong res...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,k in Faceted Project Framework while changing ...,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 9.29 seconds, 10.34 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5149789452552795, R=0.45883235335350037, F1=0.48528704047203064
Row 1, Bug ID 2: BERTScore P=0.4448677897453308, R=0.41164207458496094, F1=0.42761048674583435
Row 2, Bug ID 3: BERTScore P=0.4278130531311035, R=0.3635811507701874, F1=0.39309045672416687
Row 3, Bug ID 4: BERTScore P=0.5333415269851685, R=0.3808661699295044, F1=0.44438862800598145
Row 4, Bug ID 5: BERTScore P=0.47751566767692566, R=0.4252183437347412, F1=0.4498521387577057
Row 5, Bug ID 6: BERTScore P=0.48179057240486145, R=0.4510188400745392, F1=0.46589717268943787
Row 6, Bug ID 7: BERTScore P=0.4694525897502899, R=0.43263113498687744, F1=0.45029035210609436
Row 7, Bug ID 8: BERTScore P=0.5396839380264282, R=0.43923771381378174, F1=0.4843074679374695
Row 8, Bug ID 9: BERTScore P=0.5271546244621277, R=0.32236310839653015, F1=0.4000745415687561
Row 9, Bug ID 10: BERTScore P=0.5054269433021545, R=0.432934433221817, F1=0.4663805067539215
Row 10, Bug ID 

In [ ]:
df2 = pd.read_csv("mistral-br-ft-eval-ads.csv")
df2.head()

,bug.id,summary
0,1,"""Incorrect comments about method depreciation ..."
1,2,"""Java files are opened instead Class files dur..."
2,3,"""WTP project migration from version1 to versio..."
3,4,"""Modulefile created without workspace referenc..."
4,5,Deadlock while changing the Eclipse runtime in...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,"""Incorrect comments about method depreciation ...","""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,"""Java files are opened instead Class files dur...","""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,"""WTP project migration from version1 to versio...","""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,"""Modulefile created without workspace referenc...","""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,Deadlock while changing the Eclipse runtime in...,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 8.76 seconds, 10.96 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5874991416931152, R=0.37893348932266235, F1=0.4607110619544983
Row 1, Bug ID 2: BERTScore P=0.5705351829528809, R=0.33564436435699463, F1=0.42264676094055176
Row 2, Bug ID 3: BERTScore P=0.4864693880081177, R=0.35630887746810913, F1=0.4113380014896393
Row 3, Bug ID 4: BERTScore P=0.6082897782325745, R=0.3492377996444702, F1=0.4437214732170105
Row 4, Bug ID 5: BERTScore P=0.6613786816596985, R=0.4271138906478882, F1=0.5190370678901672
Row 5, Bug ID 6: BERTScore P=0.6635514497756958, R=0.4178679883480072, F1=0.5128017663955688
Row 6, Bug ID 7: BERTScore P=0.6126854419708252, R=0.39265429973602295, F1=0.47859159111976624
Row 7, Bug ID 8: BERTScore P=0.523894727230072, R=0.291675865650177, F1=0.37472525238990784
Row 8, Bug ID 9: BERTScore P=0.5811958312988281, R=0.3002123534679413, F1=0.39591681957244873
Row 9, Bug ID 10: BERTScore P=0.5864589214324951, R=0.35444706678390503, F1=0.44184786081314087
Row 10, Bug ID 11: 

In [ ]:
df2 = pd.read_csv("mistral-br-ft-eval-ads-with-code.csv")
df2.head()

,bug.id,summary
0,1,"""Incorrect depreaction comments added in Eclip..."
1,2,"""Java files are opened during code inspection ..."
2,3,Compatibility issue between WTP versions in re...
3,4,Legacy calls in flat component deployable crea...
4,5,"""Deadlock in Eclipse while changing runtime du..."


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,"""Incorrect depreaction comments added in Eclip...","""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,"""Java files are opened during code inspection ...","""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,Compatibility issue between WTP versions in re...,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,Legacy calls in flat component deployable crea...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,"""Deadlock in Eclipse while changing runtime du...","""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 8.73 seconds, 11.00 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.590894877910614, R=0.3696290850639343, F1=0.4547766447067261
Row 1, Bug ID 2: BERTScore P=0.6125314235687256, R=0.356939435005188, F1=0.45104318857192993
Row 2, Bug ID 3: BERTScore P=0.648949146270752, R=0.4198288321495056, F1=0.5098299980163574
Row 3, Bug ID 4: BERTScore P=0.6014280915260315, R=0.3690347671508789, F1=0.457406222820282
Row 4, Bug ID 5: BERTScore P=0.6334120631217957, R=0.44201353192329407, F1=0.5206807255744934
Row 5, Bug ID 6: BERTScore P=0.6704311370849609, R=0.413651704788208, F1=0.5116305947303772
Row 6, Bug ID 7: BERTScore P=0.637068510055542, R=0.42499393224716187, F1=0.5098574757575989
Row 7, Bug ID 8: BERTScore P=0.5240792036056519, R=0.3293321430683136, F1=0.40448519587516785
Row 8, Bug ID 9: BERTScore P=0.581293523311615, R=0.32382386922836304, F1=0.4159387946128845
Row 9, Bug ID 10: BERTScore P=0.5898119211196899, R=0.366365909576416, F1=0.45198073983192444
Row 10, Bug ID 11: BERTScore 

In [ ]:
df2 = pd.read_csv("phi-3-br-ft-eval-ads.csv")
df2.head()

,bug.id,summary
0,1,Deprecated comments and missing replacement me...
1,2,This bug was fixed in eclipse-jee-mars-RC3-win...
2,3,Runtime compatibility issue between Eclipse WT...
3,4,WTP bug causes modulefiles to be created with ...
4,5,Deadlock during project list modification. Sol...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,Deprecated comments and missing replacement me...,"""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,This bug was fixed in eclipse-jee-mars-RC3-win...,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,Runtime compatibility issue between Eclipse WT...,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,WTP bug causes modulefiles to be created with ...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,Deadlock during project list modification. Sol...,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 9.65 seconds, 9.95 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.531890332698822, R=0.38441580533981323, F1=0.4462854266166687
Row 1, Bug ID 2: BERTScore P=0.579785168170929, R=0.4700062572956085, F1=0.5191558003425598
Row 2, Bug ID 3: BERTScore P=0.5219624638557434, R=0.31027793884277344, F1=0.3891986906528473
Row 3, Bug ID 4: BERTScore P=0.5394303202629089, R=0.45561352372169495, F1=0.49399179220199585
Row 4, Bug ID 5: BERTScore P=0.4961755871772766, R=0.4245958924293518, F1=0.45760348439216614
Row 5, Bug ID 6: BERTScore P=0.4703926146030426, R=0.4321633577346802, F1=0.45046836137771606
Row 6, Bug ID 7: BERTScore P=0.5566236972808838, R=0.4970349669456482, F1=0.5251443386077881
Row 7, Bug ID 8: BERTScore P=0.589218258857727, R=0.4500303566455841, F1=0.5103034973144531
Row 8, Bug ID 9: BERTScore P=0.4945807456970215, R=0.39504319429397583, F1=0.43924349546432495
Row 9, Bug ID 10: BERTScore P=0.49074023962020874, R=0.44009774923324585, F1=0.4640413820743561
Row 10, Bug ID 11: BE

In [ ]:
df2 = pd.read_csv("phi-3-br-ft-eval-ads-with-code.csv")
df2.head()

,bug.id,summary
0,1,Deprecated comments and missing replacement fo...
1,2,Bug: eclipse-bugs/eclipse/org.osgi.framework/3...
2,3,Runtime compatibility issue between Eclipse WT...
3,4,WTP creates wrong modulefiles for binary compo...
4,5,Deadlock during project facets creation and mo...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,Deprecated comments and missing replacement fo...,"""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,Bug: eclipse-bugs/eclipse/org.osgi.framework/3...,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,Runtime compatibility issue between Eclipse WT...,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,WTP creates wrong modulefiles for binary compo...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,Deadlock during project facets creation and mo...,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 9.69 seconds, 9.90 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5003410577774048, R=0.4674273729324341, F1=0.48332449793815613
Row 1, Bug ID 2: BERTScore P=0.5267821550369263, R=0.4567421078681946, F1=0.4892682433128357
Row 2, Bug ID 3: BERTScore P=0.48024165630340576, R=0.3012634217739105, F1=0.3702579736709595
Row 3, Bug ID 4: BERTScore P=0.5597760081291199, R=0.4672259986400604, F1=0.5093308687210083
Row 4, Bug ID 5: BERTScore P=0.595730721950531, R=0.3832613527774811, F1=0.4664400517940521
Row 5, Bug ID 6: BERTScore P=0.46104416251182556, R=0.4123794436454773, F1=0.435356080532074
Row 6, Bug ID 7: BERTScore P=0.41965389251708984, R=0.39506787061691284, F1=0.4069899022579193
Row 7, Bug ID 8: BERTScore P=0.59012371301651, R=0.44468218088150024, F1=0.5071820616722107
Row 8, Bug ID 9: BERTScore P=0.4989980161190033, R=0.42619064450263977, F1=0.45972955226898193
Row 9, Bug ID 10: BERTScore P=0.4946599006652832, R=0.4304298460483551, F1=0.46031510829925537
Row 10, Bug ID 11: BERT

In [ ]:
df2 = pd.read_csv("gemma-br-ft-eval-ads.csv")
df2.head()

,bug.id,summary
0,1,"Sure, here's a summary in one sentence: The bu..."
1,2,"Sure, here's a summary in one sentence:\n\nThe..."
2,3,"Sure, here's a summary in one sentence :\n\nTh..."
3,4,"Sure, here's a summary in one sentence: The co..."
4,5,"Sure, here is a summary in one sentence :\n\nT..."


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,"Sure, here's a summary in one sentence: The bu...","""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,"Sure, here's a summary in one sentence:\n\nThe...","""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,"Sure, here's a summary in one sentence :\n\nTh...","""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,"Sure, here's a summary in one sentence: The co...","""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,"Sure, here is a summary in one sentence :\n\nT...","""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 8.78 seconds, 10.93 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5874090194702148, R=0.37298715114593506, F1=0.4562617540359497
Row 1, Bug ID 2: BERTScore P=0.5348714590072632, R=0.3950134217739105, F1=0.4544248580932617
Row 2, Bug ID 3: BERTScore P=0.5249283313751221, R=0.3855057656764984, F1=0.4445415437221527
Row 3, Bug ID 4: BERTScore P=0.6192705631256104, R=0.40954941511154175, F1=0.4930345416069031
Row 4, Bug ID 5: BERTScore P=0.5488216876983643, R=0.4160788655281067, F1=0.4733194410800934
Row 5, Bug ID 6: BERTScore P=0.6236915588378906, R=0.4671674966812134, F1=0.5341999530792236
Row 6, Bug ID 7: BERTScore P=0.6334166526794434, R=0.48384732007980347, F1=0.5486204624176025
Row 7, Bug ID 8: BERTScore P=0.5236412286758423, R=0.3537518382072449, F1=0.4222487211227417
Row 8, Bug ID 9: BERTScore P=0.5398243069648743, R=0.3413974344730377, F1=0.41827070713043213
Row 9, Bug ID 10: BERTScore P=0.5761292576789856, R=0.40561455488204956, F1=0.47606393694877625
Row 10, Bug ID 11: BE

In [ ]:
df2 = pd.read_csv("gemma-br-ft-eval-ads-with-code.csv")
df2.head()

,bug.id,summary
0,1,"Sure, here's a summary in one sentence :\n\nTh..."
1,2,The issue is related to source lookup and debu...
2,3,"Sure, here's a summary in one sentence: The pr..."
3,4,The code creates a module file without referen...
4,5,"Sure, here is a summary in one sentence:\n\nTh..."


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,"Sure, here's a summary in one sentence :\n\nTh...","""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,The issue is related to source lookup and debu...,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,"Sure, here's a summary in one sentence: The pr...","""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,The code creates a module file without referen...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,"Sure, here is a summary in one sentence:\n\nTh...","""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 8.80 seconds, 10.91 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5350518822669983, R=0.4043288826942444, F1=0.4605947732925415
Row 1, Bug ID 2: BERTScore P=0.6341440081596375, R=0.39579880237579346, F1=0.48739296197891235
Row 2, Bug ID 3: BERTScore P=0.5706484317779541, R=0.3850775361061096, F1=0.4598470628261566
Row 3, Bug ID 4: BERTScore P=0.6314877867698669, R=0.39702892303466797, F1=0.48753491044044495
Row 4, Bug ID 5: BERTScore P=0.5460790395736694, R=0.41385045647621155, F1=0.4708576500415802
Row 5, Bug ID 6: BERTScore P=0.589979887008667, R=0.4302802085876465, F1=0.4976312518119812
Row 6, Bug ID 7: BERTScore P=0.6196719408035278, R=0.44415634870529175, F1=0.5174354314804077
Row 7, Bug ID 8: BERTScore P=0.5205274820327759, R=0.35037294030189514, F1=0.41882801055908203
Row 8, Bug ID 9: BERTScore P=0.5785424113273621, R=0.3344610333442688, F1=0.42387548089027405
Row 9, Bug ID 10: BERTScore P=0.574005663394928, R=0.41699254512786865, F1=0.4830605685710907
Row 10, Bug ID 11: 

In [ ]:
df2 = pd.read_csv("qwen-br-ft-eval-ads.csv")
df2.head()

,bug.id,summary
0,1,The user is requesting to have the deprecated ...
1,2,The issue is when debugging a Java application...
2,3,The issue is that when importing projects from...
3,4,The issue is that when creating modulefiles fo...
4,5,The issue is a deadlock that occurs when the F...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,The user is requesting to have the deprecated ...,"""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,The issue is when debugging a Java application...,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,The issue is that when importing projects from...,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,The issue is that when creating modulefiles fo...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,The issue is a deadlock that occurs when the F...,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 9.37 seconds, 10.24 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5396918654441833, R=0.49979662895202637, F1=0.5189786553382874
Row 1, Bug ID 2: BERTScore P=0.5387911796569824, R=0.4611041247844696, F1=0.49692967534065247
Row 2, Bug ID 3: BERTScore P=0.633493185043335, R=0.502946138381958, F1=0.5607214570045471
Row 3, Bug ID 4: BERTScore P=0.5375904440879822, R=0.47015801072120667, F1=0.5016181468963623
Row 4, Bug ID 5: BERTScore P=0.5959100723266602, R=0.5347452163696289, F1=0.563673198223114
Row 5, Bug ID 6: BERTScore P=0.5746555328369141, R=0.4807680547237396, F1=0.5235357880592346
Row 6, Bug ID 7: BERTScore P=0.5154666304588318, R=0.4972682297229767, F1=0.5062038898468018
Row 7, Bug ID 8: BERTScore P=0.5987945199012756, R=0.46675896644592285, F1=0.5245963335037231
Row 8, Bug ID 9: BERTScore P=0.5341967344284058, R=0.435155987739563, F1=0.47961676120758057
Row 9, Bug ID 10: BERTScore P=0.5805540680885315, R=0.48080992698669434, F1=0.5259951949119568
Row 10, Bug ID 11: BERTSc

In [ ]:
df2 = pd.read_csv("qwen-br-ft-eval-ads-with-code.csv")
df2.head()

,bug.id,summary
0,1,The user is asking to have the deprecated meth...
1,2,The issue is when debugging Java applications ...
2,3,The issue is that when importing projects from...
3,4,The issue is that when a flat component is dep...
4,5,A deadlock occurs in Eclipse when the FacetedP...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,The user is asking to have the deprecated meth...,"""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,The issue is when debugging Java applications ...,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,The issue is that when importing projects from...,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,The issue is that when a flat component is dep...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,A deadlock occurs in Eclipse when the FacetedP...,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 9.48 seconds, 10.13 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.5418466329574585, R=0.5061811804771423, F1=0.5234070420265198
Row 1, Bug ID 2: BERTScore P=0.500336766242981, R=0.4541085660457611, F1=0.47610312700271606
Row 2, Bug ID 3: BERTScore P=0.5918059349060059, R=0.4979977309703827, F1=0.5408644080162048
Row 3, Bug ID 4: BERTScore P=0.553316056728363, R=0.472882479429245, F1=0.5099471211433411
Row 4, Bug ID 5: BERTScore P=0.49203282594680786, R=0.4815903306007385, F1=0.4867555797100067
Row 5, Bug ID 6: BERTScore P=0.532383918762207, R=0.48634082078933716, F1=0.5083219408988953
Row 6, Bug ID 7: BERTScore P=0.6112591028213501, R=0.5383906960487366, F1=0.572515606880188
Row 7, Bug ID 8: BERTScore P=0.5346282124519348, R=0.45945313572883606, F1=0.494198203086853
Row 8, Bug ID 9: BERTScore P=0.5294721126556396, R=0.4429168105125427, F1=0.4823422133922577
Row 9, Bug ID 10: BERTScore P=0.577930212020874, R=0.4703274667263031, F1=0.5186061859130859
Row 10, Bug ID 11: BERTScore P

In [ ]:
df2 = pd.read_csv("deepseek-br-ft-eval-ads.csv")
df2.head()

,bug.id,summary
0,1,The depreciated method 'getResourceString' in ...
1,2,The issue of Java source code not being shown ...
2,3,The bug in Eclipse's WTP (Web Tools Platform) ...
3,4,The legacy code creates a module file without ...
4,5,Deadlock issue in Faceted Project Framework wh...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,The depreciated method 'getResourceString' in ...,"""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,The issue of Java source code not being shown ...,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,The bug in Eclipse's WTP (Web Tools Platform) ...,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,The legacy code creates a module file without ...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,Deadlock issue in Faceted Project Framework wh...,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 9.27 seconds, 10.35 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.594287633895874, R=0.5278686285018921, F1=0.5591124892234802
Row 1, Bug ID 2: BERTScore P=0.5987864136695862, R=0.48578399419784546, F1=0.5363982915878296
Row 2, Bug ID 3: BERTScore P=0.5070536136627197, R=0.4516284465789795, F1=0.4777388572692871
Row 3, Bug ID 4: BERTScore P=0.5527183413505554, R=0.47845807671546936, F1=0.51291424036026
Row 4, Bug ID 5: BERTScore P=0.6033680438995361, R=0.5630093216896057, F1=0.5824904441833496
Row 5, Bug ID 6: BERTScore P=0.6448561549186707, R=0.537371814250946, F1=0.5862278938293457
Row 6, Bug ID 7: BERTScore P=0.5848655700683594, R=0.5058466196060181, F1=0.5424937605857849
Row 7, Bug ID 8: BERTScore P=0.6018646955490112, R=0.45712971687316895, F1=0.5196065902709961
Row 8, Bug ID 9: BERTScore P=0.5731889009475708, R=0.43841132521629333, F1=0.49682173132896423
Row 9, Bug ID 10: BERTScore P=0.6198604702949524, R=0.47522076964378357, F1=0.537988543510437
Row 10, Bug ID 11: BERTSco

In [ ]:
df2 = pd.read_csv("deepseek-br-ft-eval-ads-with-code.csv")
df2.head()

,bug.id,summary
0,1,The depreciated method 'getResourceString' in ...
1,2,The issue is with the inspection of class file...
2,3,The bug in Eclipse's Web Tools Platform (WTP) ...
3,4,Binary EARs with web modules are creating modu...
4,5,Deadlock issue in Faceted Project Framework wh...


In [ ]:
df2 = df2.merge(df, on='bug.id', how='left')
df2.head()

,bug.id,summary,bug_report,golden_summary
0,1,The depreciated method 'getResourceString' in ...,"""(223734) eclipse - incorrect deprecation comm...",according to the javadoc in platform.getresour...
1,2,The issue is with the inspection of class file...,"""(346116) eclipse - Java files open when inspe...","i found a regression from 342 to 362, similar ..."
2,3,The bug in Eclipse's Web Tools Platform (WTP) ...,"""(201329) eclipse - [hotbug] runtime compatibi...","due to the changes in bug# 137825, projects cr..."
3,4,Binary EARs with web modules are creating modu...,"""(312336) eclipse - flatcomponentdeployable le...",i noticed that in the members calls for flatco...
4,5,Deadlock issue in Faceted Project Framework wh...,"""(221376) eclipse - deadlock at the facetedpro...","in an adopter product, while executing tests s..."


In [ ]:
from bert_score import score


def evaluate_bert(golden_summaries, generated_summaries):
    P, R, F1 = score(generated_summaries, golden_summaries, lang="en", model_type="microsoft/deberta-xlarge-mnli", verbose=True)
    return P.tolist(), R.tolist(), F1.tolist()


valid_rows = df2.dropna(subset=["golden_summary", "summary"])


bert_precision, bert_recall, bert_f1 = evaluate_bert(valid_rows["golden_summary"].tolist(), valid_rows["summary"].tolist())


df2.loc[valid_rows.index, "bert_precision"] = bert_precision
df2.loc[valid_rows.index, "bert_recall"] = bert_recall
df2.loc[valid_rows.index, "bert_f1"] = bert_f1

for idx, (p, r, f) in enumerate(zip(bert_precision, bert_recall, bert_f1)):
    print(f"Row {valid_rows.index[idx]}, Bug ID {valid_rows.iloc[idx]['bug.id']}: BERTScore P={p}, R={r}, F1={f}")

avg_p = sum(bert_precision) / len(bert_precision)
avg_r = sum(bert_recall) / len(bert_recall)
avg_f1 = sum(bert_f1) / len(bert_f1)

print(f"\nAverage BERTScore - Precision: {avg_p:.4f}, Recall: {avg_r:.4f}, F1: {avg_f1:.4f}")

calculating scores...
computing bert embedding.


  0%|          | 0/3 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/2 [00:00<?, ?it/s]

done in 9.69 seconds, 9.91 sentences/sec
Row 0, Bug ID 1: BERTScore P=0.6147545576095581, R=0.5161381363868713, F1=0.5611464977264404
Row 1, Bug ID 2: BERTScore P=0.5664591193199158, R=0.465217649936676, F1=0.5108708143234253
Row 2, Bug ID 3: BERTScore P=0.6043741703033447, R=0.4984760582447052, F1=0.546340823173523
Row 3, Bug ID 4: BERTScore P=0.5924128293991089, R=0.49223592877388, F1=0.5376982688903809
Row 4, Bug ID 5: BERTScore P=0.5274132490158081, R=0.509017825126648, F1=0.5180522799491882
Row 5, Bug ID 6: BERTScore P=0.5507968068122864, R=0.5037639141082764, F1=0.5262315273284912
Row 6, Bug ID 7: BERTScore P=0.5499918460845947, R=0.5048397779464722, F1=0.52644944190979
Row 7, Bug ID 8: BERTScore P=0.6115778684616089, R=0.461242139339447, F1=0.5258766412734985
Row 8, Bug ID 9: BERTScore P=0.5305061936378479, R=0.420728862285614, F1=0.4692831039428711
Row 9, Bug ID 10: BERTScore P=0.5993983745574951, R=0.469809353351593, F1=0.5267506837844849
Row 10, Bug ID 11: BERTScore P=0.51968